In [2]:
import sys
sys.path.insert(1, './scripts')

from libraries import *
from functions import *
from fitting import *
from util import *

%matplotlib qt
data[0]

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


{'name': 'V1649180633_1.ir_rc20final.cub',
 'sequenceid': 'S59',
 'observationid': 'VIMS_129TI_REGMAP001_ISS',
 'latitude': 6.527894053385,
 'longitude': 117.23227585525,
 'phase': 4.35186034428,
 'incidence': 71.368914431631,
 'emission': 72.074559422824,
 'distance': 25208.69808185,
 'lineres': 38679.377390385,
 'sampres': 12885.433750399,
 'scposx': 15904.022205055,
 'scposy': 19558.581460496,
 'scposz': 49.274092100503,
 'sunposx': 993649050.83181,
 'sunposy': 1013825175.7289,
 'sunposz': 93586120.333315,
 'ettime': 323758866.57842,
 'starttime': '2010-04-05 16:59:19.48',
 'exposureduration': 220.0,
 'spectrum': array([ 0.0763505 ,  0.113136  ,  0.138922  ,  0.146524  ,  0.126     ,
         0.0936954 ,  0.0795553 ,  0.0763359 ,  0.0885651 ,  0.111927  ,
         0.131274  ,  0.140332  ,  0.13276   ,  0.100027  ,  0.0593369 ,
         0.0431969 ,  0.0295527 ,  0.0313845 ,  0.042422  ,  0.0622391 ,
         0.0827431 ,  0.0844537 ,  0.0981416 ,  0.109633  ,  0.124524  ,
         0.0

### Why are some pixels unreasonable?


There a few exploding spectra but a few that dip under 0. Verified that it is consistent with CSV output from SQL query. Work around this by filtering pixels out according to fit parameters and raw spectra values.

In [3]:
bad_pixels = []
filtered_data = filter_data(data)

for i,entry in enumerate(filtered_data):
    if np.max(entry['spectrum']) > 1.0 or np.min(entry['spectrum']) < -.1:
        bad_pixels.append((entry,i))

len(bad_pixels)

{'name': 'V1753535313_1.ir_rc20final.cub', 'sequenceid': 'S79', 'observationid': 'VIMS_195TI_HIRES003_PRIME', 'latitude': -14.192410526012, 'longitude': 113.08447714133, 'phase': 79.247331641247, 'incidence': 68.628679969023, 'emission': 26.016340587574, 'distance': 7368.7025357152, 'lineres': 1965.8545564098, 'sampres': 2466.4575678517, 'scposx': -2735.0425878636, 'scposy': 5677.0692141084, 'scposz': -3819.4507773567, 'sunposx': -1380094499.4225, 'sunposy': 147413363.40772, 'sunposz': 488789535.78307, 'ettime': 428112953.54426, 'starttime': '2013-07-26 12:12:41.994', 'exposureduration': 160.0, 'spectrum': array([ 0.0404514 ,  0.0663658 ,  0.086725  ,  0.0854239 ,  0.0644469 ,
        0.0462576 ,  0.0390803 ,  0.0374057 ,  0.048135  ,  0.0629884 ,
        0.0766593 ,  0.0830114 ,  0.0691124 ,  0.0468681 ,  0.0240458 ,
        0.0202946 ,  0.0128574 ,  0.016177  ,  0.021298  ,  0.032576  ,
        0.0397113 ,  0.0422687 ,  0.0508583 ,  0.059501  ,  0.0663442 ,
        0.042854  ,  0.026

Total 94816 entries removed


0

In [4]:
def plot_spectra(entry):
    plt.plot(vims_wave, entry)

In [5]:
plot_spectra(filtered_data[32131]['spectrum'])

### Plot bump parameters against viewing geometry

Looking for empirical corrections that can be applied.

In [34]:
def plot_bump_parameters(data, sample_every = 1):
    
    from mpl_toolkits.mplot3d import Axes3D

    # should also consider filtering out entries where bump isn't present? 
    
    fig, axs = plt.subplots(3,3)
    data_as_list = {'incidence': [], 
                    'emission': [], 
                    'phase': [], 
                    'bump_max': [],
                    'bump_max_channel': [],
                    'slope': [],
                    'constant': [],
                    'ettime': []}
    
    for i, entry in enumerate(data):
        if i % sample_every == 0 and entry['bump_info'] != {}:
            
            data_as_list['incidence'].append(entry['incidence'])
            data_as_list['emission'].append(entry['emission'])
            data_as_list['phase'].append(entry['phase'])
            data_as_list['bump_max'].append(entry['bump_info']['bump_max'])
            data_as_list['slope'].append(entry['bump_info']['polyfit'][0])
            data_as_list['constant'].append(entry['bump_info']['polyfit'][1])
            data_as_list['bump_max_channel'].append(entry['bump_info']['bump_max_channel'])
            data_as_list['ettime'].append(entry[''])
    axs[0, 0].scatter(data_as_list['incidence'], data_as_list['bump_max'])
    axs[0, 0].set_title('Bump max vs Incidence')
    
    axs[0, 1].scatter(data_as_list['emission'], data_as_list['bump_max'])
    axs[0, 1].set_title('Bump max vs Emission')
    
    axs[0, 2].scatter(data_as_list['phase'], data_as_list['bump_max'])
    axs[0, 2].set_title('Bump max vs Phase')
    
#     axs[1, 0].scatter(data_as_list['incidence'], data_as_list['bump_max_channel'])
#     axs[1, 0].set_title('Bump channel vs Incidence')

#     axs[1, 1].scatter(data_as_list['emission'], data_as_list['bump_max_channel'])
#     axs[1, 1].set_title('Bump channel vs Emission')

#     axs[1, 2].scatter(data_as_list['phase'], data_as_list['bump_max_channel'])
#     axs[1, 2].set_title('Bump channel vs Phase')

    axs[1, 0].scatter(data_as_list['incidence'], data_as_list['slope'])
    axs[1, 0].set_title('Bump slope vs Incidence')
    
    axs[1, 1].scatter(data_as_list['emission'], data_as_list['slope'])
    axs[1, 1].set_title('Bump slope vs Emission')
    
    axs[1, 2].scatter(data_as_list['phase'], data_as_list['slope'])
    axs[1, 2].set_title('Bump slope vs Phase')
    
    axs[2, 0].scatter(data_as_list['incidence'], data_as_list['constant'])
    axs[2, 0].set_title('Bump constant vs Incidence')
    
    axs[2, 1].scatter(data_as_list['emission'], data_as_list['constant'])
    axs[2, 1].set_title('Bump constant vs Emission')
    
    axs[2, 2].scatter(data_as_list['phase'], data_as_list['constant'])
    axs[2, 2].set_title('Bump Constant vs Phase')
    
    plt.tight_layout(pad=0.00001)

    fig2 = plt.figure()
    ax = fig2.add_subplot(111, projection = '3d')
    
    ax.scatter(data_as_list['bump_max'], data_as_list['incidence'], data_as_list['emission'])
    ax.set_xlabel('bump max')
    ax.set_ylabel('incidence')
    ax.set_zlabel('emission')

In [35]:
plot_bump_parameters(filtered_data, 10)